<a href="https://colab.research.google.com/github/alvivar/llama2-googlecolab/blob/main/Llama2_from_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Huggingface

In [ ]:
# Classic transformers
!pip install huggingface-hub
!pip install transformers

# Optimizers
!pip install accelerate
!pip install xformers

# Fine tunning
!pip install trl
!pip install peft
!pip install bitsandbytes
!git clone https://github.com/lvwerra/trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Auth

In [ ]:
# Tokens
# https://huggingface.co/settings/tokens

# You can also use this clic to login on a terminal.
# !huggingface-cli login

access_key = ''
with open('access_key.txt', 'r') as file:
    access_key = file.read()

import huggingface_hub
huggingface_hub.login(access_key)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Downloads if needed

In [ ]:
import requests
from tqdm import tqdm

def download_file_with_progress(url, destination_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(destination_path, 'wb') as file, tqdm(
        desc=destination_path,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as progress_bar:
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            progress_bar.update(len(data))

url = "https://www.dropbox.com/s/dkj1dyc2ro7053h/pico8.psd?dl=1"
destination_path = "pico8.psd"

download_file_with_progress(url, destination_path)

pico8.psd: 100%|██████████| 55.9k/55.9k [00:00<00:00, 15.9MB/s]


# Prepare the pipeline from the pretrained model

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# The prompt

In [ ]:
sequences = pipeline(
    'Explain humanity from the point of view of a god.\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Explain humanity from the point of view of a god.
Humanity is a species of great complexity and potential. As a god, I have watched them for eons, marveling at their strengths and failures. They are capable of great beauty and great cruelty, of kindness and great selfishness. They are, in short, a species of great wonder.
At their best, humans create and innovate, pushing the boundaries of what is possible. They have built great cities and civilizations, harnessed the power of nature, and explored the farthest reaches of space. They have created art and music that brings me joy, and literature that delves deep into the human experience. They have, in short, achieved greatness.
But they are also prone to flaws and weaknesses. They are quick to conflict and violence, often over petty differences. They are blinded by their own passions and biases, and are slow to see the perspectives of others. They are, in short, imperfect.
Despite these flaws, however, I remain fascinated by hum

# Finetunning with SFTTrainer from trl

## Interesting datasets

- timdettmers/openassistant-guanaco
- roneneldan/TinyStories
- Fraser/mnist-text-small

In [ ]:
# Small fix to make sure UTF-8 is the default locale.
# From https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name roneneldan/TinyStories \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2

2023-07-27 02:34:59.363140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100% 2/2 [00:08<00:00,  4.43s/it]
Repo card metadata block was not found. Setting CardData to empty.



Extracting data files: 100% 2/2 [00:00<00:00, 1002.46it/s]
Generating train split: 2119719 examples [00:06, 302893.91 examples/s]
Generating validation split: 21990 examples [00:00, 301091.46 examples/s]
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: User